# e.g. 18: SODA

Calculate wet/area file for use in other scripts.

Calculate t16,20,22d for use in other scripts.

# last edited 30 August 2018

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [11]:
print('BEGIN')

%matplotlib inline

__file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

import numpy as np
import numpy.ma as ma
import inspect
import pickle
import bz2
import glob
import socket
import re
import os
import netCDF4
import shutil
from area import area #https://github.com/scisco/area

CRED = '\033[91m'
CEND = '\033[0m'

hostname=socket.gethostname()

print('hostname=',hostname)

if(re.match('raijin',hostname)):
  print('this is rajin')
  rundir='/short/v14/mac599/cafepp/rundir'
  topdir=''
elif(re.match('oa-3.-cdc',hostname)):
  print('this is vm32')
  rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
  topdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6'

os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

from n_data_funcs import n_data_funcs

from decadal_diag import \
  box_indices, \
  diag_isothetaoNc, \
  calc_isoN, \
  shade_2d_latlon, \
  shade_2d_simple, \
  smooth, \
  plot_xy, \
  plot_box_indices

model='SODA'

print('END')

BEGIN
hostname= oa-35-cdc
this is vm32
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
END


In [16]:
print('BEGIN')

'''
This bit of code will generate SODA cell area/wet data to be used for example with wwv calculation.

'''
  
Clobber=True
#Clobber=False

soda_ifil='/OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/soda3.7.2_mn_ocean_reg_1980.nc'

ifh = netCDF4.Dataset(soda_ifil,'r')

temp = ifh.variables['temp'][0,0,]
soda_lat = ifh.variables['yt_ocean']
soda_lon = ifh.variables['xt_ocean']

temp = temp/temp
temp.set_fill_value(0)
soda_wet = temp.filled()

# print(temp)
# print(type(temp))
# print(temp.shape)
# print(temp.filled())

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

soda_ofil='/OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/wetarea_soda3.7.2_mn_ocean_reg.nc'

print('Output file: '+soda_ofil)

if(os.path.exists(soda_ofil)):
  if(Clobber): os.remove(soda_ofil)

ofh = netCDF4.Dataset(soda_ofil, 'w', format='NETCDF4') #might choose NETCDF3 or NETCDF4_CLASSIC if want to use MFDataset
  
latD = ofh.createDimension('grid_y_T', soda_lat.size)
lonD = ofh.createDimension('grid_x_T', soda_lon.size)

latV = ofh.createVariable('grid_y_T','f4',('grid_y_T',))
lonV = ofh.createVariable('grid_x_T','f4',('grid_x_T',))

wetV = ofh.createVariable('wet', 'f8', ('grid_y_T','grid_x_T',), fill_value=1e20)
areaV = ofh.createVariable('area_T', 'f8', ('grid_y_T','grid_x_T',), fill_value=1e20)

latV.units='degree_n'
latV.long_name='Nominal Latitude of T-cell center'
latV.cartesian_axis='Y'
                                          
lonV.units='degree_e'
lonV.long_name='Nominal Longitude of T-cell center'
lonV.cartesian_axis='X'

wetV.units='none'
wetV.long_name='land/sea flag (0=land) for T-cell'
                                          
areaV.units='m2'
areaV.long_name='Area of T_cell'

lat_vals=soda_lat[:]
lon_vals=soda_lon[:]

lat_min_vals=np.append((1.5*lat_vals[0] - 0.5*lat_vals[1]), (lat_vals[0:-1] + lat_vals[1:])/2)
lat_max_vals=np.append((lat_vals[0:-1] + lat_vals[1:])/2, 1.5*lat_vals[-1] - 0.5*lat_vals[-2])
lat_vals_bounds=np.column_stack((lat_min_vals, lat_max_vals))

lon_min_vals=np.append((1.5*lon_vals[0] - 0.5*lon_vals[1]), (lon_vals[0:-1] + lon_vals[1:])/2)
lon_max_vals=np.append((lon_vals[0:-1] + lon_vals[1:])/2, 1.5*lon_vals[-1] - 0.5*lon_vals[-2])
lon_vals_bounds=np.column_stack((lon_min_vals, lon_max_vals))

print('lat_vals_bounds.shape=',lat_vals_bounds.shape)
print('lon_vals_bounds.shape=',lon_vals_bounds.shape)

soda_area = ma.zeros((soda_lat.size, soda_lon.size),dtype=np.float64)

# obj = {'type':'Polygon','coordinates':[[[-180,-90],[-180,90],[180,90],[180,-90],[-180,-90]]]}
# >>> area(obj)

for j in range(soda_lat.size):
  for i in range(soda_lon.size):
    #print(lat_vals_bounds[j,0],lon_vals_bounds[i,0])
    lat_min=lat_vals_bounds[j,0]
    lat_max=lat_vals_bounds[j,1]
    lon_min=lon_vals_bounds[i,0]
    lon_max=lon_vals_bounds[i,1]
    soda_area[j,i] = area({'type':'Polygon','coordinates':[[[lon_min,lat_min],[lon_min,lat_max],[lon_max,lat_max],[lon_max,lat_min],[lon_min,lat_min]]]})
    #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

latV[:] = soda_lat[:]
lonV[:] = soda_lon[:]

wetV[:] = soda_wet
areaV[:] = soda_area

total_area= np.sum(np.sum(soda_area))

print('total surface area=',total_area/(1000*1000)/1e6,'million km^2')

ofh.close()

print('END')

BEGIN
Output file: /OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/wetarea_soda3.7.2_mn_ocean_reg.nc
lat_vals_bounds.shape= (330, 2)
lon_vals_bounds.shape= (720, 2)
502.4984001148269
END


In [12]:
print('BEGIN')

'''
This bit of code will generate SODA t20 etc files, so these can be read in rather than recalculated each time. Could put this is a separate notebook?
'''

# print('soda_temp_annual_climatology_from_monthly.shape=',soda_temp_annual_climatology_from_monthly.shape)
# print('soda_temp_annual_anomaly_from_monthly.shape=',soda_temp_annual_anomaly_from_monthly.shape) 
#print('soda_t20d_annual_climatology_from_monthly.shape=',soda_t20d_annual_climatology_from_monthly.shape)

#raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

#soda_ifil='/OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/soda3.7.2_mn_ocean_reg_1980.nc'

soda_units = 'days since 1980-01-01 00:00:00'
soda_calendar = 'proleptic_gregorian'
  
soda_ifils=sorted(glob.glob('/OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/soda3.7.2_mn_ocean_reg_????.nc'))

ifh0 = netCDF4.Dataset(soda_ifils[0], 'r')
ifhN = netCDF4.MFDataset(soda_ifils, 'r')

#temp = ifh0.variables['temp'][0,0,]
soda_lat = ifh0.variables['yt_ocean']
soda_lon = ifh0.variables['xt_ocean']
soda_lev = ifh0.variables['st_ocean']
soda_time = ifhN.variables['time']

print(soda_time)

soda_date_time_stamp=netCDF4.num2date(soda_time[:], soda_units, soda_calendar)

Clobber=True
#Clobber=False

isotherm_values=['20','16','22']
isotherm_values=['25']

for isotherm_value in isotherm_values:
  
  ybeg=soda_date_time_stamp[0].year
  yend=soda_date_time_stamp[-1].year
  
  soda_ofil='/OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/t'+isotherm_value+'d_soda3.7.2_mn_ocean_reg_'+str('{0:04d}'.format(ybeg))+'01-'+str('{0:04d}'.format(yend))+'12.nc'

  print('Output file: '+soda_ofil)
  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))
  
  if(os.path.exists(soda_ofil)):
    if(Clobber): os.remove(soda_ofil)

  ofh = netCDF4.Dataset(soda_ofil, 'w', format='NETCDF4') #might choose NETCDF3 or NETCDF4_CLASSIC if want to use MFDataset

  timeD = ofh.createDimension('time', None)
  latD = ofh.createDimension('yt_ocean', soda_lat.size)
  lonD = ofh.createDimension('xt_ocean', soda_lon.size)

  timeV = ofh.createVariable('time','f8',('time',))
  latV = ofh.createVariable('yt_ocean', 'f4', ('yt_ocean',))
  lonV = ofh.createVariable('xt_ocean', 'f4', ('xt_ocean',))

  varV = ofh.createVariable('t'+isotherm_value+'d', 'f4', ('time','yt_ocean','xt_ocean',), fill_value=1e20)
  
  timeV.units = soda_units
  timeV.calendar = soda_calendar

  latV.units='degrees_N'
  latV.standard_name='latitude'
  latV.long_name='tcell latitude'

  lonV.units='degrees_E'
  lonV.standard_name='longitude'
  lonV.long_name='tcell longitude'

  varV.units='m'
  varV.long_name='Depth of the '+isotherm_value+'C isotherm'
  varV.standard_name='depth_of_the_'+isotherm_value+'C_isotherm'
  varV.missing_value=1e20

  latV[:] = soda_lat[:]
  lonV[:] = soda_lon[:]

  #first_value=calc_isoN(ifhN.variables['temp'][0,], int(isotherm_value), soda_lev, 24, 60, 240, diag=True) #limit to 500m, 45s-45n

  for time in range(soda_time[:].size):
  #for time in range(1): #write out one time for testing
    print('time=',time)
    varV[time,] = calc_isoN(ifhN.variables['temp'][time,], int(isotherm_value), soda_lev, 24, 60, 240, diag=True) #limit to 500m, 45s-45n
    #varV[time,] = first_value
    timeV[time] = soda_time[time]

  ofh.close()
  
print('END')

BEGIN
<class 'netCDF4._netCDF4._Variable'>
float64 time('time',)
    standard_name: time
    long_name: time
    units: days since 1980-01-01 00:00:00
    calendar: standard
    axis: T
    cell_methods: time: mean
unlimited dimensions = ('time',)
current size = (444,)

Output file: /OSM/CBR/OA_DCFP/data2/col414/SODA/dsrs.atmos.umd.edu/DATA/soda3.7.2/REGRIDED/ocean/t25d_soda3.7.2_mn_ocean_reg_198001-201612.nc
time= 0


jupyter_notebook:86: UserWarning: WARNING: missing_value cannot be safely cast to variable dtype
/OSM/CBR/OA_DCFP/work/col414/cafepp/decadal_diag.py:2713: RuntimeWarning: divide by zero encountered in double_scalars
  newdata[y,x] = (data[xabove[y,x],y,x] - value) / (data[xabove[y,x],y,x] - data[xbelow[y,x],y,x]) * (levs[xbelow[y,x]] - levs[xabove[y,x]]) + levs[xabove[y,x]]
/OSM/CBR/OA_DCFP/work/col414/cafepp/decadal_diag.py:2713: RuntimeWarning: invalid value encountered in double_scalars
  newdata[y,x] = (data[xabove[y,x],y,x] - value) / (data[xabove[y,x],y,x] - data[xbelow[y,x],y,x]) * (levs[xbelow[y,x]] - levs[xabove[y,x]]) + levs[xabove[y,x]]
/OSM/CBR/OA_DCFP/work/col414/cafepp/decadal_diag.py:2713: UserWarning: Warning: converting a masked element to nan.
  newdata[y,x] = (data[xabove[y,x],y,x] - value) / (data[xabove[y,x],y,x] - data[xbelow[y,x],y,x]) * (levs[xbelow[y,x]] - levs[xabove[y,x]]) + levs[xabove[y,x]]


KeyboardInterrupt: 